In [21]:
import serial
import time
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [22]:
sampling_rate = 32#Hz
sampling_interval = 1.0 / sampling_rate  # Convert Hz to seconds

# Set up the serial connection (Update 'COM3' as per your system)
ser = serial.Serial('COM6', 115200, timeout=1)
time.sleep(2)  # Allow time for connection

# Initialize data storage
data = []
start_time = time.time()

print(f"Collecting data at {sampling_rate} Hz for 60 seconds...")

while time.time() - start_time < 60:
    try:
        line = ser.readline().decode('utf-8').strip()  # Read and decode serial data
        if line:  # Ensure the line is not empty
            flipped_value = float(line)  # Convert to float
            timestamp = time.time() - start_time  # Capture timestamp
            data.append((timestamp, flipped_value))  # Store timestamp and value
            print(f"{timestamp:.2f}s: {flipped_value:.2f}")  # Print real-time float data
            # print(f"{datetime.datetime.now().hour}:{datetime.datetime.now().minute}:{datetime.datetime.now().second}:{datetime.datetime.now().microsecond}-> {flipped_value:.2f}")  # Print real-time float data

        time.sleep(sampling_interval)  # Control the sampling rate

    except ValueError:
        print("Non-numeric data received, skipping...")  # Handle conversion errors
    except Exception as e:
        print("Error:", e)
        break

ser.close()  # Close serial connection

# Convert to DataFrame and save to CSV
df = pd.DataFrame(data, columns=['Time', 'BVP'])
# df.to_csv('BVP_values.csv', index=False)
print(f"Data collection complete.' at {sampling_rate} Hz.")

Non-numeric data received, skipping...
2.13s: 27577.00
2.16s: 27577.00
2.19s: 27577.00
2.23s: 27577.00
2.26s: 27577.00
2.29s: 27577.00
2.32s: 27577.00
2.35s: 27577.00
2.39s: 27577.00
2.42s: 27577.00
2.45s: 27577.00
2.48s: 27577.00
2.51s: 27577.00
2.55s: 27577.00
2.58s: 27577.00
2.61s: 27577.00
2.64s: 27577.00
2.67s: 27577.00
2.70s: 27577.00
2.74s: 27577.00
2.77s: 27577.00
2.80s: 27577.00
2.83s: 27577.00
2.87s: 27577.00
2.90s: 27577.00
2.93s: 27577.00
2.96s: 27577.00
2.99s: 27577.00
3.03s: 27577.00
3.06s: 27577.00
3.09s: 27577.00
3.12s: 27577.00
3.15s: 27577.00
3.19s: 27577.00
3.22s: 27577.00
3.25s: 27577.00
3.28s: 27577.00
3.31s: 27577.00
3.35s: 27577.00
3.38s: 27577.00
3.41s: 27577.00
3.44s: 27577.00
3.48s: 27577.00
3.51s: 27577.00
3.54s: 27577.00
3.57s: 27577.00
3.60s: 27577.00
3.64s: 27577.00
3.67s: 27577.00
3.70s: 27577.00
3.73s: 27577.00
3.77s: 27577.00
3.80s: 27577.00
3.83s: 27577.00
3.86s: 27577.00
3.90s: 27577.00
3.93s: 27577.00
3.96s: 27577.00
3.99s: 27577.00
4.02s: 27577.00
4

In [23]:
df

,Time,BVP
0,2.127196,27577.0
1,2.161383,27577.0
2,2.193635,27577.0
3,2.225590,27577.0
4,2.257423,27577.0
...,...,...
1790,59.827443,72158.0
1791,59.859779,72192.0
1792,59.891674,72192.0
1793,59.923982,72173.0


In [ ]:
t = df['Time']
y = df['BVP']

plt.figure(figsize=(9,3))
plt.plot(t[-960:],y[-960:], color='blue',linewidth=2, label='BVP')
plt.xlabel('Time(s)')
plt.ylabel('BVP')
plt.title('Raw BVP Signal Collected from MAX 30105 Sensor')
# plt.legend()
plt.show()

In [ ]:
from scipy.signal import butter, filtfilt

# Define Butterworth High-Pass Filter
def butter_highpass(cutoff, fs, order=2):
    nyq = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

# Apply the filter
def apply_highpass_filter(data, cutoff=0.5, fs=32, order=2):
    b, a = butter_highpass(cutoff, fs, order)
    y_filtered = filtfilt(b, a, data)
    return y_filtered

# Set parameters
sampling_rate = 32 
cutoff_freq = 0.5 

# Apply high-pass filter
y_filtered = apply_highpass_filter(y, cutoff=cutoff_freq, fs=sampling_rate)

# Plot Raw and Filtered Signals
plt.figure(figsize=(9, 3))

# plt.plot(t[-960:], y[-960:], color='blue', linewidth=2, label='Raw BVP', alpha=0.5)
plt.plot(t[-960:], y_filtered[-960:], color='red', linewidth=2, label='Filtered BVP')

plt.xlabel('Time (s)')
plt.ylabel('BVP')
plt.title('Raw and High-Pass Filtered BVP Signal (MAX30105 Sensor)')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model.h5')
print('Model Loaded')

In [ ]:
X_test = y_filtered[-960:]
X_test.shape

In [ ]:
X_test = y_filtered[-960:]
# y_test_mean = np.mean(y_test)
# y_test_sd = np.std(y_test)
# y_test_norm = (y_test-y_test_mean)/y_test_sd

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_test.reshape(-1, 1))

plt.figure(figsize=(9,3))
plt.plot(t[-960:],X_scaled, color='darkorange',linewidth=2, label='BVP')
plt.xlabel('Time(s)')
plt.ylabel('BVP')
plt.title('Normalized BVP Signal Collected from MAX 30105 Sensor')
# plt.legend()
plt.show()

In [ ]:
#Prediction'
y_pred_prob = model.predict(X_scaled.reshape(-1, 960, 1))

y_pred = np.argmax(y_pred_prob, axis=1)
for pred in y_pred:
    if pred == 1:
        print("Stressed")
    else:
        print("Normal")

In [ ]:
print(y_pred_prob)